In [223]:
%reset

In [224]:
import pandas as pd
import numpy as np

In [225]:
df_btc = pd.read_csv('data/btc-usd-max.csv')
df_ir = pd.read_csv('data/10year_real_interest_rate.csv')
df_gdp = pd.read_csv('data/us_gdp_t.txt')
df_inflation = pd.read_csv('data/us_inflation_rate.txt')

## Data preprocessing

In [226]:
df_btc = df_btc[(df_btc['snapped_at'] >= '2019-12-31') & (df_btc['snapped_at'] <= '2024-01-01')]
df_ir = df_ir[(df_ir['DATE'] >= '2020-01-01') & (df_ir['DATE'] <= '2024-01-01')]
df_gdp = df_gdp[df_gdp['Date'] >= '2019-12-31']
df_inflation = df_inflation[df_inflation['Date'] >= '2019-12-31']

In [227]:
df_btc['snapped_at'] = pd.to_datetime(df_btc['snapped_at']).dt.tz_localize(None)
df_btc.columns = ['date', 'btc_price', 'btc_market_cap', 'btc_volume_24hr']
df_btc.dtypes

date               datetime64[ns]
btc_price                 float64
btc_market_cap            float64
btc_volume_24hr           float64
dtype: object

In [228]:
df_btc['btc_supply'] = df_btc['btc_market_cap'] / df_btc['btc_price']
df_btc.drop(['btc_market_cap'], axis=1, inplace=True)

In [229]:
df_btc.set_index('date', inplace=True)
weekly_spread = df_btc['btc_price'].resample('W').apply(lambda x: x.max() - x.min())
df_btc['btc_weekly_spread'] = df_btc.index.to_series().map(weekly_spread)
df_btc.reset_index(inplace=True)

In [230]:
df_ir['DATE'] = pd.to_datetime(df_ir['DATE'])
df_ir.columns = ['date', 'us_real_interest_rate_percent']
df_ir.dtypes

date                             datetime64[ns]
us_real_interest_rate_percent           float64
dtype: object

In [231]:
df_gdp['Date'] = pd.to_datetime(df_gdp['Date'])
df_gdp['Value'] = df_gdp['Value'].str.rstrip('T').astype(float)
df_gdp['Value'] = df_gdp['Value'] * 1e12
df_gdp.columns = ['date', 'us_gdp_usd']
df_gdp.dtypes

date          datetime64[ns]
us_gdp_usd           float64
dtype: object

In [232]:
df_inflation['Date'] = pd.to_datetime(df_inflation['Date'])
df_inflation.columns = ['date', 'us_inflation_rate_percent']
df_inflation.dtypes

date                         datetime64[ns]
us_inflation_rate_percent           float64
dtype: object

## Merge dataframes

In [233]:
df_btc_weekly = df_btc.set_index('date')
df_btc_weekly = df_btc_weekly.resample('W').mean()
df_btc_weekly.reset_index(inplace=True)
df_btc_weekly.head(10)

,date,btc_price,btc_volume_24hr,btc_supply,btc_weekly_spread
0,2020-01-05,7207.485803,2.226211e+10,1.814566e+07,387.523699
1,2020-01-12,7900.463817,3.163291e+10,1.812270e+07,830.540922
2,2020-01-19,8633.594520,3.913943e+10,1.815467e+07,796.254711
3,2020-01-26,8558.372633,2.991802e+10,1.817600e+07,376.152958
4,2020-02-02,9179.121879,4.520534e+10,1.818911e+07,914.076837
5,2020-02-09,9550.608454,6.358609e+10,1.820205e+07,710.742377
6,2020-02-16,10147.125407,5.710329e+10,1.821032e+07,438.776342
7,2020-02-23,9765.613191,4.569227e+10,1.822225e+07,525.648236
8,2020-03-01,9116.596085,4.041274e+10,1.823555e+07,1393.727005
9,2020-03-08,8866.565895,4.536157e+10,1.824718e+07,568.042817


In [234]:
df_btc_weekly['year_month'] = df_btc_weekly['date'].dt.to_period('M')
df_ir['year_month'] = df_ir['date'].dt.to_period('M')
df_gdp['year_month'] = df_gdp['date'].dt.to_period('M')
df_inflation['year_month'] = df_inflation['date'].dt.to_period('M')

df_week = pd.merge(df_btc_weekly, df_ir.drop(['date'], axis=1), on='year_month', how='left')
df_week = pd.merge(df_week, df_gdp.drop(['date'], axis=1), on='year_month', how='left')
df_week = pd.merge(df_week, df_inflation.drop(['date'], axis=1), on='year_month', how='left')
df_week.drop(['year_month'], axis=1, inplace=True)
df_week.reset_index()
df_week.head(10)

,date,btc_price,btc_volume_24hr,btc_supply,btc_weekly_spread,us_real_interest_rate_percent,us_gdp_usd,us_inflation_rate_percent
0,2020-01-05,7207.485803,2.226211e+10,1.814566e+07,387.523699,0.505096,2.202000e+13,2.49
1,2020-01-12,7900.463817,3.163291e+10,1.812270e+07,830.540922,0.505096,2.202000e+13,2.49
2,2020-01-19,8633.594520,3.913943e+10,1.815467e+07,796.254711,0.505096,2.202000e+13,2.49
3,2020-01-26,8558.372633,2.991802e+10,1.817600e+07,376.152958,0.505096,2.202000e+13,2.49
4,2020-02-02,9179.121879,4.520534e+10,1.818911e+07,914.076837,0.315986,2.212000e+13,2.33
5,2020-02-09,9550.608454,6.358609e+10,1.820205e+07,710.742377,0.315986,2.212000e+13,2.33
6,2020-02-16,10147.125407,5.710329e+10,1.821032e+07,438.776342,0.315986,2.212000e+13,2.33
7,2020-02-23,9765.613191,4.569227e+10,1.822225e+07,525.648236,0.315986,2.212000e+13,2.33
8,2020-03-01,9116.596085,4.041274e+10,1.823555e+07,1393.727005,-0.127837,2.098000e+13,1.54
9,2020-03-08,8866.565895,4.536157e+10,1.824718e+07,568.042817,-0.127837,2.098000e+13,1.54


In [235]:
# df_week.to_csv('data/df_week.csv')